In [1]:
import simpleFoam_tools as sft
import pandas as pd
import numpy as np

In [2]:
remove = 1
if remove:
    sft.remove_run_files(".")

In [3]:
domain_name = "vti_bent_x"
vti_path = f"constant/geometry/{domain_name}.vti"
stl_path = f"constant/triSurface/{domain_name}.stl"

In [4]:
# radii = 10
shape = (100, 100, 100)

In [5]:
scale=1e-6
dp = 1e-3
stl = f"{domain_name}.stl"
refinement = 0
factor_mesh = 1
# location_in_mesh = (25, 25, 25)
location_in_mesh = (50, 70, 20)
boundary_type = "symmetryPlane" # "Wall" or "symmetryPlane"
end_time = 300
write_interval = 50

In [6]:
sft.vti_to_stl(vti_path, stl_path)

Running command: pvpython /home/h09435ap/porePermFoam/simpleFoam_tools/paraview_stl.py /home/h09435ap/porePermFoam/constant/geometry/vti_bent_x.vti /home/h09435ap/porePermFoam/constant/triSurface/vti_bent_x.stl
hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
✅ Wrote ASCII STL: /home/h09435ap/porePermFoam/constant/triSurface/vti_bent_x.stl


In [7]:
mesh_resolution = tuple(x * factor_mesh for x in shape)
blockMeshDict_path    = "system/blockMeshDict"
controlDict_path      = "system/controlDict"
p_field_path          = "0/p"
U_field_path          = "0/U"
snappyHexMeshDict_path = "system/snappyHexMeshDict"
sft.generate_blockMeshDict(shape, mesh_resolution, blockMeshDict_path, boundary=boundary_type)
sft.generate_snappyHexMeshDict(location_in_mesh, stl, snappyHexMeshDict_path, refinement=refinement)
sft.generate_controlDict(controlDict_path, end_time=end_time, write_interval=write_interval)
sft.generate_pressure_field(p_field_path, dp=dp, boundary=boundary_type)
sft.generate_velocity_field(U_field_path, boundary=boundary_type)

Generated blockMeshDict at: system/blockMeshDict
Generated snappyHexMeshDict at: system/snappyHexMeshDict
Generated controlDict at: system/controlDict
Generated p at: 0/p with boundary symmetryPlane
Generated U at: 0/U with boundary symmetryPlane


In [8]:
sft.run_simplefoam(".", scale=scale)


>>> blockMesh

/*---------------------------------------------------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  2412                                  |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
Build  : _f00230df-20250528 OPENFOAM=2412 patch=250528 version=2412
Arch   : "LSB;label=32;scalar=64"
Exec   : blockMesh
Date   : Aug 07 2025
Time   : 22:57:12
Host   : e-10aux32876g5
PID    : 2041873
I/O    : uncollated
Case   : /home/h09435ap/porePermFoam
nProcs : 1
trapFpe: Floating point exception trapping enabled (FOAM_SIGFPE).
fileModificationChecking : Monitoring run-time modified files using timeStampMaster (file

In [9]:
df_q = pd.read_csv("q_in.csv")
df_q.head()

,time,flowRate_phi,area
0,0.0,-1.307160e-13,1.837690e-09
1,50.0,-1.480210e-13,1.837690e-09
2,100.0,-1.515560e-13,1.837690e-09
3,150.0,-1.523350e-13,1.837690e-09
4,200.0,-1.525110e-13,1.837690e-09


In [10]:
phi = sft.phi_vti(vti_path)
print(f"porosity: {phi:.02%}")

porosity: 24.81%


In [11]:
A = shape[1] * shape[2] * scale**2
miu = 1e-3
L = shape[0] * scale
Q = np.abs(df_q["flowRate_phi"].iloc[-1])  
dp = dp * 1e3 # convert to Pa
k = Q * miu * L / (A * dp)
print(f"permeability: {k:.02e} m^2")
print(f"permeability: {k * 1.01324997e15:.02f} md")

permeability: 1.53e-12 m^2
permeability: 1545.73 md
